## Filter the data by language and aggregate the useful values

In [ ]:
# import necessary modules
import pandas as pd
from os import scandir

In [ ]:
data_folder = './data/'
dir = scandir(data_folder)
# Boolean flag to diffrentiate the first file
first = True
for elem in dir:
    df = pd.read_csv(data_folder+elem.name, encoding='UTF-8', compression='gzip', parse_dates=['date'], infer_datetime_format=True)
    df = df[df.language == 'en']
    groupby = df.groupby(pd.Grouper(key='date', freq='d'))
    counts = groupby.agg('count').reset_index()[['date', 'tweet']]
    sums = groupby.agg('sum').reset_index()[['date', 'likes_count', 'replies_count', 'retweets_count']]
    counts = counts.merge(sums, on='date')

    if first:
        first_str = elem.name.split('_')[0]
        first = False
        agg = counts.copy()
    else:
        agg = agg.merge(counts, how='outer', on='date', suffixes=(None, '_'+elem.name.split('_')[0])).fillna(0)

new_cols = [agg.columns[0]]
new_cols.extend([x + f'_{first_str}' for x in agg.columns[1:5]])
new_cols.extend(agg.columns[5:])
agg.columns = new_cols

agg.to_csv(data_folder+'aggregated.csv',) #compression is no longer needed

In [27]:
agg.head()

,Unnamed: 0,date,tweet_abu sayyaf,likes_count_abu sayyaf,replies_count_abu sayyaf,retweets_count_abu sayyaf,tweet_agro,likes_count_agro,replies_count_agro,retweets_count_agro,...,replies_count_tehrik-i-taliban pakistan,retweets_count_tehrik-i-taliban pakistan,tweet_weapons-grade,likes_count_weapons-grade,replies_count_weapons-grade,retweets_count_weapons-grade,tweet_yemen,likes_count_yemen,replies_count_yemen,retweets_count_yemen
0,0,2012-01-01,6.0,0.0,0.0,0.0,79,0.0,10.0,5.0,...,0.0,0.0,17,1.0,5.0,2.0,1583,60.0,105.0,544.0
1,1,2012-01-02,0.0,0.0,0.0,0.0,180,0.0,42.0,6.0,...,0.0,0.0,12,0.0,8.0,4.0,1647,59.0,129.0,572.0
2,2,2012-01-03,2.0,0.0,0.0,0.0,186,5.0,40.0,13.0,...,0.0,0.0,9,0.0,2.0,2.0,1461,51.0,138.0,437.0
3,3,2012-01-04,2.0,0.0,0.0,0.0,211,1.0,27.0,3.0,...,0.0,0.0,11,1.0,0.0,5.0,1680,57.0,114.0,586.0
4,4,2012-01-05,9.0,0.0,0.0,0.0,204,0.0,41.0,8.0,...,0.0,0.0,11,0.0,2.0,0.0,1694,43.0,120.0,436.0


This operation gives us the above aggregated Dataframe containing the number of tweets, likes, replies and retweets per day per topic